In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fake-news/submit.csv
/kaggle/input/fake-news/train.csv
/kaggle/input/fake-news/test.csv


In [2]:
# Load data
train = pd.read_csv('/kaggle/input/fake-news/train.csv')
test = pd.read_csv('/kaggle/input/fake-news/test.csv')

# Data explore

In [3]:
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
print(train['label'].value_counts())

label
1    10413
0    10387
Name: count, dtype: int64


In [5]:
train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
train = train.dropna(subset=['text'])

In [7]:
train['sentence_length'] = train['text'].apply(len)
print(train['sentence_length'].describe())

count     20761.000000
mean       4552.715380
std        5130.563491
min           1.000000
25%        1628.000000
50%        3361.000000
75%        6275.000000
max      142961.000000
Name: sentence_length, dtype: float64


# Preprocessing

In [ ]:
! pip install nltk

In [ ]:
import string
import nltk
nltk.download('punkt')

In [ ]:
train.drop_duplicates(subset='text', keep='first', inplace=True)

In [ ]:
train['text'] = train['text'].str.replace(
'['+string.punctuation+']', '', regex=True)

In [ ]:
train['text'] = train['text'].str.lower()

In [13]:
train = train.dropna(subset=['text'])
train['text'] = train['text'].astype(str).apply(nltk.word_tokenize)

# SVM

In [14]:
! pip install XGBoost

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from xgboost import XGBClassifier

In [16]:
X_train = train['text']
y_train = train['label']

In [17]:
vectorizer = TfidfVectorizer()
X_train = X_train.apply(' '.join)
X_train = vectorizer.fit_transform(X_train)

In [18]:
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train, test_size=0.2, stratify=y_train, random_state=42
)

In [19]:
# clf =XGBClassifier(
#                 n_estimators=  100,  # number of trees
#                 max_depth = 10,  # maximum depth of the trees
#                 learning_rate =  0.01,
# )
# clf.fit(X_train_split, y_train_split)

In [20]:
# # Training performance
# y_train_pred = clf.predict(X_train_split)
# print(classification_report(y_train_split, y_train_pred))

In [21]:
# # Training performance
# y_val_pred = clf.predict(X_val_split)
# print(classification_report(y_val_split, y_val_pred))

# FINE TUNING 2


In [22]:
# clf =XGBClassifier(
#                 n_estimators=  50,  # number of trees
#                 max_depth = 10,  # maximum depth of the trees
#                 learning_rate =  0.001,
# )
# clf.fit(X_train_split, y_train_split)

In [23]:
# # Training performance
# y_train_pred = clf.predict(X_train_split)
# print(classification_report(y_train_split, y_train_pred))

In [24]:
# # Training performance
# y_val_pred = clf.predict(X_val_split)
# print(classification_report(y_val_split, y_val_pred))

### fine tuning 3

In [25]:
# clf =XGBClassifier(
#                 n_estimators=  100,  # number of trees
#                 max_depth = 10,  # maximum depth of the trees
#                 learning_rate =  0.001,
# )
# clf.fit(X_train_split, y_train_split)

In [26]:
# # Training performance
# y_train_pred = clf.predict(X_train_split)
# print(classification_report(y_train_split, y_train_pred))

In [27]:
# # Training performance
# y_val_pred = clf.predict(X_val_split)
# print(classification_report(y_val_split, y_val_pred))

In [28]:
# clf =XGBClassifier(
#                 n_estimators=  200,  # number of trees
#                 max_depth = 10,  # maximum depth of the trees
#                 learning_rate =  0.001,
# )
# clf.fit(X_train_split, y_train_split)

In [29]:
# # Training performance
# y_train_pred = clf.predict(X_train_split)
# print(classification_report(y_train_split, y_train_pred))

In [30]:
# # Training performance
# y_val_pred = clf.predict(X_val_split)
# print(classification_report(y_val_split, y_val_pred))

In [31]:
clf =XGBClassifier(
                n_estimators=  200,  # number of trees
                max_depth = 10,  # maximum depth of the trees
                learning_rate =  0.01,
                tree_method="hist", device="cuda"
)
clf.fit(X_train_split, y_train_split)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [32]:
# Training performance
y_train_pred = clf.predict(X_train_split)
print(classification_report(y_train_split, y_train_pred))

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [12:46:32] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


              precision    recall  f1-score   support

           0       0.55      1.00      0.71      8309
           1       0.98      0.15      0.27      7999

    accuracy                           0.58     16308
   macro avg       0.76      0.58      0.49     16308
weighted avg       0.76      0.58      0.49     16308



In [33]:
# Training performance
y_val_pred = clf.predict(X_val_split)
print(classification_report(y_val_split, y_val_pred))

              precision    recall  f1-score   support

           0       0.55      1.00      0.71      2078
           1       0.97      0.15      0.26      2000

    accuracy                           0.58      4078
   macro avg       0.76      0.57      0.48      4078
weighted avg       0.76      0.58      0.49      4078



In [34]:
clf =XGBClassifier(
                n_estimators=  200,  # number of trees
                max_depth = 12,  # maximum depth of the trees
                learning_rate =  0.01,
)
clf.fit(X_train_split, y_train_split)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=12, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [35]:
# Training performance
y_train_pred = clf.predict(X_train_split)
print(classification_report(y_train_split, y_train_pred))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      8309
           1       0.96      0.99      0.98      7999

    accuracy                           0.98     16308
   macro avg       0.98      0.98      0.98     16308
weighted avg       0.98      0.98      0.98     16308



In [36]:
# Training performance
y_val_pred = clf.predict(X_val_split)
print(classification_report(y_val_split, y_val_pred))

              precision    recall  f1-score   support

           0       0.96      0.91      0.93      2078
           1       0.91      0.96      0.93      2000

    accuracy                           0.93      4078
   macro avg       0.93      0.93      0.93      4078
weighted avg       0.93      0.93      0.93      4078



In [ ]:
clf =XGBClassifier(
                n_estimators=  200,  # number of trees
                max_depth = 15,  # maximum depth of the trees
                learning_rate =  0.01,
)
clf.fit(X_train_split, y_train_split)

In [41]:
def preprocess_data(data):
    data = data.str.replace('['+string.punctuation+']', '', regex=True)
    data= data.str.lower()
    data = data.astype(str).apply(nltk.word_tokenize)
    data = data.apply(' '.join)
    data = vectorizer.transform(data)
    return data

In [42]:
X_test = preprocess_data(test['text'])
id_test = test['id']
y_test_pred = clf.predict(X_test)

In [43]:
# Create a DataFrame
submission_df = pd.DataFrame({'id': id_test, 'label': y_test_pred})
# Save the DataFrame to a CSV file
submission_df.to_csv('/kaggle/working/xg2_boost_submission.csv', index=False)


In [40]:
from joblib import dump
dump(clf, 'svm_model.joblib')

['svm_model.joblib']